This code is in-progress. Its purpose is to read in the tractography data into a dataframe and convert it into a format compatible for machine learning (wide format) -- Random Forest

In [ ]:
import csv
import numpy as np
import os
import random as rd
import re
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import theano.tensor as T

from IPython.core.pylabtools import figsize
from matplotlib import pyplot as plt
from nipy import load_image, save_image
from nipy.core.api import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier 
from sklearn import linear_model
from sklearn.model_selection import RandomizedSearchCV, cross_validate, StratifiedKFold
from sklearn.metrics import plot_roc_curve, auc 
from sklearn.base import clone
from subprocess import call

In [ ]:
os.chdir('.')

In [ ]:
#don't use this anymore, but keeping incase I want it later
def convert_ca(date_string):
    #input format: MM/DD/YY
    month = int(date_string.split('/')[0])
    day = int(date_string.split('/')[1])
    year = int(date_string.split('/')[2])
    #return format: YYYYMMDD
    return (year+2000)*10000 + month*100 + day
    

def load_tract_data(tractdata):
    filename = tractdata.split('/')[-1]
    
    StudyID = int(filename.split('_')[0])
    MR_num = filename.split('_')[1]
    Scan_num = filename.split('_')[2]
    if Scan_num[-4:]=='.txt':
        Scan_num = Scan_num[:-4]
    data = pd.read_csv(tractdata, sep='\t')
#     data = data.drop(['Unnamed: 21'], axis=1) #old way of doing the below
    data = data.iloc[: , :-1] #get rid of empty last column
    data = data.rename(columns={'Tract Name': 'field'})
    data['field'] = data['field'].str.replace(' ', '_')
    data = pd.melt(data, id_vars=['field'],
            var_name='tract', value_name='value')
    data['tract.field'] = data['tract'] + '.' + data['field']
    data['StudyID'] = StudyID
    data['MR_num'] = MR_num
    data['Scan_num'] = Scan_num
    
    return data


In [ ]:
Eddy_corrected = '/data/Reback-DTI/resources/stats/Eddy_corrected/'
Noncorrected = '/data/Reback-DTI/resources/stats/Noncorrected'
Batch1 = os.path.join(Eddy_corrected, 'Batch1')
Batch2 = os.path.join(Eddy_corrected, 'Batch2')
Batch3 = os.path.join(Eddy_corrected, 'Batch3')

Batch_dir  = Batch3
input_Dir  = os.path.join(Batch_dir, 'Input_dir')
output_dir = os.path.join(Batch_dir, 'Output_dir')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    

In [ ]:
stat_files = [os.path.join(input_Dir, X) for X in os.listdir(input_Dir) if X[-4:]=='.txt']
patient_datas = []
for file in stat_files:
    data = load_tract_data(file)
    patient_datas.append(data)
diffusion_data = pd.concat(patient_datas)
wide_data = diffusion_data.pivot(index=['StudyID', 'MR_num', 'Scan_num'], columns=['tract.field'], values='value')
wide_data = wide_data.reset_index()
# wide_data

In [ ]:
# wide_data.to_csv('/data/Reback-DTI/wide_data.csv', index=True, sep=',')

In [ ]:
def load_clinical_data(src):
    #note, i had to "clean" some of the data manually before using it in this program
    # I only converted y=1 and n=0 and ? = ' '
    clinical_data_file = src
    clinical_data = pd.read_csv(clinical_data_file, sep=',')
    clinical_data = clinical_data.dropna(how='all')
    clinical_data['Study ID'] = clinical_data['Study ID'].astype(int)
    #necessary to merge with data elsewhere
    clinical_data = clinical_data.rename(columns={"Study ID": "StudyID"})
    clinical_data = clinical_data.drop(['Unnamed: 20'], axis=1)
    clinical_data.columns = clinical_data.columns.str.replace(' ', '_')
    #convert date to an number
    #clinical_data['Date CA'] = clinical_data['Date CA'].map(lambda x: convert_ca(x))
    clinical_data['Date_CA'] = pd.to_datetime(clinical_data['Date_CA'], dayfirst=False)
    clinical_data['Date_CA'] = clinical_data['Date_CA'].astype(int) / 10**9
    #REPLACE EMPTY CELLS WITH NaN
    clinical_data = clinical_data.replace(' ', np.NaN)
    for var in clinical_data.columns:
        try:
            clinical_data[var] = clinical_data[var].astype('float64')
        except Exception as e:
            print(f'{str(e)} for column: {var}')
    clinical_data = clinical_data.fillna(clinical_data.mean())
    return clinical_data

clinical_data_src = '/data/Reback-DTI/resources/clinicaldata/clinicaldata.csv'
clinical_data = load_clinical_data(clinical_data_src)
clinical_data

In [ ]:
clin_stat_data = wide_data.merge(clinical_data, on ='StudyID', how = 'inner')
#Note: the following are subjects without associated clinical data
#notIncl = wide_data[~wide_data.StudyID.isin(clin_stat_data.StudyID)]
clin_stat_data.to_csv(os.path.join(output_dir, 'clin_stat_data.csv'), index=False, sep=',')
clin_stat_data

In [ ]:
#keep the closest post-CA scan
def load_MR_Date_data(MR_Date_src):
    MR_Date_data = pd.read_csv(MR_Date_src, sep=',')
    MR_Date_data = MR_Date_data.rename(columns={"MR ID": "MR_ID"})
    MR_Date_data = MR_Date_data.rename(columns={"Date": "ScanDate"})
    #convert date to an number
    MR_Date_data['ScanDate'] = pd.to_datetime(MR_Date_data['ScanDate'], dayfirst=False)
    MR_Date_data['ScanDate'] = MR_Date_data['ScanDate'].astype(int)/ 10**9
    #MR_Date_data['ScanDate'] = MR_Date_data['ScanDate'].map(lambda x: convert_ca(x)) 
    return MR_Date_data

def filterDuplicates(in_data):
    #get MR dates
    MR_Date_src = '/data/Reback-DTI/resources/clinicaldata/Reback-MRIs.csv'   
    MR_Date_data = load_MR_Date_data(MR_Date_src)
    
    #add MR dates to running data
    out_data = in_data
    out_data['MR_ID'] = out_data['StudyID'].map(str).str.zfill(4) + '_' + out_data['MR_num'].map(str)
    out_data = out_data.merge(MR_Date_data, on ='MR_ID', how = 'inner')
    
    #remove scans that happens before the CA
    out_data['TCA2MR'] = out_data['ScanDate'] - out_data['Date_CA']
    out_data1 = out_data[out_data['TCA2MR'] < 0]
    out_data = out_data[out_data['TCA2MR'] >= 0]
    #The following are MRIs taken before CA
    #print(out_data1)
    
    
    #sort by time and only keep the scans closest to the CA
    out_data = out_data.sort_values(by=['StudyID','TCA2MR'], ascending=True)
    #The following are duplicate scans
    #duplicate = out_data[out_data.duplicated('StudyID')]
    #print(duplicate)
    out_data = out_data.drop_duplicates(subset = ['StudyID'], keep='first')

    
    return out_data

RF_data = filterDuplicates(clin_stat_data)
RF_data.to_csv(os.path.join(output_dir, 'RF_data.csv'), index=False, sep=',')
RF_data

In [ ]:
#adding the group to the whole frame
#temporary to run ANOVA
RF_data.loc[(RF_data['Age_y'] >= 0) & (RF_data['Age_y'] < 1), 'AgeGroup'] = '0' 
RF_data.loc[(RF_data['Age_y'] >= 1) & (RF_data['Age_y'] < 5), 'AgeGroup'] = '1' 
RF_data.loc[(RF_data['Age_y'] >= 5) , 'AgeGroup'] = '2'

In [ ]:
RF_data.to_csv(os.path.join(output_dir, 'RF_ageStrat.csv'), index=False, sep=',')

In [ ]:
dataAge0_1 = RF_data[(RF_data['Age_y'] >= 0) & (RF_data['Age_y'] < 1)]
dataAge1_5 = RF_data[(RF_data['Age_y'] >= 1) & (RF_data['Age_y'] < 5)]
dataAge5_  = RF_data[(RF_data['Age_y'] >= 5)]
dataAge1_  = RF_data[(RF_data['Age_y'] >= 1)]
dataAge0_5 = RF_data[(RF_data['Age_y'] >= 0) & (RF_data['Age_y'] < 5)]
dataAge0_2 = RF_data[(RF_data['Age_y'] >= 0) & (RF_data['Age_y'] < 2)]

base_dir = output_dir
strat_dir = os.path.join(base_dir, 'strats')

all_ages_dir = os.path.join(strat_dir, 'all_ages/')
if not os.path.exists(all_ages_dir):
    os.makedirs(all_ages_dir)
age0_1_dir   = os.path.join(strat_dir, 'ages0_1/')
if not os.path.exists(age0_1_dir):
    os.makedirs(age0_1_dir)
age1_5_dir   = os.path.join(strat_dir, 'ages1_5/')
if not os.path.exists(age1_5_dir):
    os.makedirs(age1_5_dir)
age5__dir    = os.path.join(strat_dir, 'ages5_/')
if not os.path.exists(age5__dir):
    os.makedirs(age5__dir)
age1__dir    = os.path.join(strat_dir, 'ages1_/')
if not os.path.exists(age1__dir):
    os.makedirs(age1__dir)
age0_5_dir   = os.path.join(strat_dir, 'ages0_5/')
if not os.path.exists(age0_5_dir):
    os.makedirs(age0_5_dir)
age0_2_dir   = os.path.join(strat_dir, 'ages0_2/')
if not os.path.exists(age0_2_dir):
    os.makedirs(age0_2_dir)

RF_data.to_csv(os.path.join(all_ages_dir,   'all_ages.csv'),   index=True, sep=',')
dataAge0_1.to_csv(os.path.join(age0_1_dir,  'ages0_1.csv'),    index=True, sep=',')
dataAge1_5.to_csv(os.path.join(age1_5_dir,  'ages1_5.csv'),    index=True, sep=',')
dataAge5_.to_csv(os.path.join(age5__dir,    'ages5_.csv'),     index=True, sep=',')
dataAge1_.to_csv(os.path.join(age1__dir,    'ages1_.csv'),     index=True, sep=',')
dataAge0_5.to_csv(os.path.join(age0_5_dir,  'ages0_5.csv'),    index=True, sep=',')
dataAge0_2.to_csv(os.path.join(age0_2_dir,  'ages0_2.csv'),    index=True, sep=',')

********************************
**Now begins the code necessary to make/run the RF
********************************

In [ ]:
def evaluate_model(test_predictions, test_labels, test_probs,
                   train_predictions, train_labels, train_probs, model_name):
    baseline = {}
    baseline['recall'] = recall_score(test_labels, [1 for _ in range(len(test_labels))])
    baseline['precision'] = precision_score(test_labels, [1 for _ in range(len(test_labels))])
    baseline['roc'] = 0.5
    
    test_results = {}
    
    test_results['recall'] = recall_score(test_labels, test_predictions)
    test_results['precision'] = precision_score(test_labels, test_predictions)
    test_results['roc'] = roc_auc_score(test_labels, test_probs)
    
    train_results = {}
    
    train_results['recall'] = recall_score(train_labels, train_predictions)
    train_results['precision'] = precision_score(train_labels, train_predictions)
    train_results['roc'] = roc_auc_score(train_labels, train_probs)
    
    for metric in ['recall', 'precision', 'roc']:
        print(f'{metric.capitalize()} Baseline: {round(baseline[metric], 2)} Test: {round(test_results[metric], 2)}, Train: {round(train_results[metric], 2)}')
        
    base_fpr, base_tpr, _ = roc_curve(test_labels, [1 for _ in range(len(test_labels))])
    model_fpr, model_tpr, _ = roc_curve(test_labels, test_probs)
    
    plt.figure(figsize=(8, 6))
    plt.rcParams['font.size'] = 16
    
    #Plot both curves
    plt.plot(base_fpr, base_tpr, 'b', label='baseline')
    plt.plot(model_fpr, model_tpr, 'r', label='model')
    plt.legend();
    plt.xlabel('False Positive Rate');
    plt.ylabel('True Positive Rate');
    plt.title('ROC Curve')  
    plt.savefig(f'/data/Reback-DTI/resources/stats/MODEL-{model_name}_AUROC')
    plt.close()
    
def find_best_model_and_train_on_split(predictors, outcome, test_size=0.2, iterations=1000):


    train, test, train_labels, test_labels = train_test_split(predictors, outcome, 
                                                          stratify = outcome, 
                                                         test_size = test_size)

    print(f"Train size: {train.shape}")
    print(f"Test size: {test.shape}")


# Hyperparameter grid
    param_grid = {
        'n_estimators': np.linspace(50, 1000, num=100).astype(int),
        'max_depth': [None] + list(np.linspace(5, 30, num=30).astype(int)),
        'max_features': ['auto', 'sqrt', None] + list(np.arange(0.3, 1, 0.1)),
        'max_leaf_nodes': [None] + list(np.linspace(5, 50, 50).astype(int)),
        'min_samples_split': [2, 5, 10],
        'bootstrap': [True]
    }
    

    # Estimator for use in random search
    estimator = RandomForestClassifier()

    # Create the random search model
    rs = RandomizedSearchCV(estimator, param_grid, n_jobs = -1, 
                            scoring = 'roc_auc', cv = 2, 
                            n_iter = iterations, verbose = 1)

    # Fit 
    rs.fit(train, train_labels)
    best_params = rs.best_params_
    
    print(f'Best parameters: {best_params}')

    best_model = rs.best_estimator_

    n_nodes = []
    max_depths = []

    for ind_tree in best_model.estimators_:
        n_nodes.append(ind_tree.tree_.node_count)
        max_depths.append(ind_tree.tree_.max_depth)

    print(f'Average number of nodes {int(np.mean(n_nodes))}')
    print(f'Average maximum depth {int(np.mean(max_depths))}')

    train_rf_predictions = best_model.predict(train)
    train_rf_probs = best_model.predict_proba(train)[:,1]

    test_rf_predictions = best_model.predict(test)
    test_rf_probs = best_model.predict_proba(test)[:,1]
    
    print(train_rf_predictions)
    print(train_labels)
    print(train_rf_probs)
    print(test_rf_predictions)
    print(test_labels)
    print(test_rf_probs)
    evaluate_model(test_rf_predictions, test_labels, test_rf_probs,
                   train_rf_predictions, train_labels, train_rf_probs, 'TEST_RF')
    
    fi_model = pd.DataFrame({'feature': list(predictors.columns),
                             'importance': best_model.feature_importances_}).sort_values('importance', ascending=False)
    
    return best_params, best_model, fi_model


In [ ]:
outcomes = ['mortality_6mos', 'pcpcgp_6mo']
outcome_data = RF_data[outcomes]
predictors_data = RF_data.drop(outcomes, axis = 1) 

In [ ]:
#best_params, best_model, fi_model = find_best_model_and_train_on_split(predictors_data, outcome_data[outcomes[1]])
#fi_model.to_csv('/data/Reback-DTI/feature_imprtances.csv', index=True, sep=',')

In [ ]:
import datetime
datetime_object = datetime.datetime.now()
print(datetime_object)